In [ ]:
import yaml
import frontmatter

In [ ]:
from collections import defaultdict
from pathlib import Path

educator_keys = [
    "mentor",
    "instructor",
    "author",
    "facilitator",
    "material",  # deprecated for author
]

educator_key_mapping = {"material": "author"}


def get_educators(base_dir: Path) -> dict[str, int]:
    profile_files = [
        f for f in base_dir.glob("*.md") if not "template" in f.name.casefold()
    ]
    educator_count = defaultdict(int)
    for f in profile_files:
        dct = frontmatter.load(f)
        for k in educator_keys:
            roles = [r.casefold() for r in dct.get("training_roles", [])]
            if k in roles:
                educator_count[k] += 1
    return educator_count

In [ ]:
get_educators(Path("/Users/fuchur/Documents/22/git_sync/hsf.github.io/_profiles/"))

In [ ]:
import git

repo = git.Repo("/Users/fuchur/Documents/22/git_sync/hsf.github.io/")

In [ ]:
path = "_profiles"

educator_counts = defaultdict(list)
times = []

for commit in repo.iter_commits(paths=path):
    try:
        traversable = (commit.tree / "_profiles").traverse()
    except (KeyError, IndexError, ValueError):
        continue
    try:
        educator_count = defaultdict(int)
        for entry in traversable:
            content = repo.git.show("{}:{}".format(commit.hexsha, entry.path))
            try:
                dct = frontmatter.loads(content)
            except Exception:  # ParserError
                pass
            keys = dct.get("training_roles", [])
            if keys is None:
                continue
            roles = [r.casefold() for r in keys]
            for k in educator_keys:
                if k in roles:
                    educator_count[educator_key_mapping.get(k, k)] += 1
                else:
                    # ensure key is present
                    educator_count[educator_key_mapping.get(k, k)] += 0
        print(commit, educator_count)
        for k, v in educator_count.items():
            educator_counts[k].append(v)
        times.append(commit.authored_date)
    except IndexError:
        continue

In [ ]:
from datetime import date

dt_times = [date.fromtimestamp(t) for t in times]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
educator_counts.keys()

In [ ]:
plt.style.use(
    "https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle"
)
# plt.style.use('fivethirtyeight')

_, ax = plt.subplots()
for k, vs in educator_counts.items():
    print(k, vs)
    ax.plot(dt_times, vs, label=k)
ax.legend(frameon=False)